# Dissociation Dynamics of H2+ in XUV and IR laser fields

The aim of this programming project is to retrace the main steps in an IR-laser-induced vibrational break-up of an XUV-ionized $H_2$-molecule, not from an experimentalists but a computational point of view. In the following, we analyze the XUV-induced ionization process in four steps to study the break-up dynamics and obtain similar results to
[[1]](https://doi.org/10.1103/PhysRevA.93.012507). 

<img src="imgs/titelbild.png" alt="Drawing" style="width: 800px;"/>

At first, the vibrational eigenstates of the $H_2^+$-molecule in its electronic ground state are determined. Second, we will depict the wave packet propagation without the IR laser field. Next, we will simulate the time evolution in the time-dependent potential with its avoided crossing behaviour, as the IR laser field couples the bound and unbound Born-Oppenheimer surfaces. Finally, we will scan the time delay of the IR pulse and consider the resulting proton momentum distribution, since the $H_2^+$ breaks into $H$ and a $H^+$ which can also be detected experimentally [[1]](https://doi.org/10.1103/PhysRevA.93.012507).

If not stated differently - especially $\tau$ given in fs in most plots - we will use atomic units throughout the whole description of this programming project.

In [ ]:
# This module contains the project specific methods, such as split-step method the laser field potential, etc.
import methods

import numpy as np
import numpy.linalg as npLA
import scipy.sparse as ss
import scipy.sparse.linalg as ssLA

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

## 0. Configuration & Preprocessing

For the reasoning on the final choice of grid size, see description of programming project, section 4. The next cell prepares all given files on the same grid, linear interpolation is due to the linear or nearly linear behaviour (see eq. 4 [[2]](https://doi.org/10.1063/1.475800) for $dipole\_coupling$) used. 

In [ ]:
# global variables
dx = 0.05  # constant for fineness of grid, could be refined but only by interpolating all .dat files (see methods file)

# load given potential and wavefunction data
dipole_coupling = methods.load_data("dipole_coupling.dat")
wave_function = methods.load_data("H2nuclwf.dat")
pot_uneven = methods.load_data("H2p_pot_ungerade.dat")
pot_even = methods.load_data("H2p_pot_gerade.dat")

# plot raw data for exploration)
plt.subplots_adjust(top=1.5, bottom=-1, left=0, right=2, hspace=0.3, wspace=0.5)

for i in np.arange(0,4,1):
    plt.subplot(221 + i)
    E = [dipole_coupling, wave_function, pot_even, pot_uneven]
    E1 = ["dipole_coupling", "H2 nucl wavefct.", "H2p pot even", "H2p pot uneven"]
    plt.plot(E[i][:,0],E[i][:,1])
    plt.xlim([0,10])
    
    plt.title("{}".format(E1[i]))
    plt.xlabel('Distance (atomic units)')
    plt.ylabel('$E$')

dipole_coupling, wave_function, pot_uneven, pot_even = methods.preprocess_data(dipole_coupling, wave_function, 
                                                                               pot_uneven, pot_even)

# normalize wave function
wave_function[:,1] /= npLA.norm(wave_function[:,1])

## 1. Find the vibrational eigenstates in the H2+ ground state potential

#### Hamiltonian for the vibrational eigenstates
We want to solve the eigenvalue problem

$$
\left[-\frac{1}{2 \mu} \partial_{x}^2 + V(x)\right] \phi(x) = E \cdot\phi(x)
$$

of the oscillating protons with reduced mass $\mu = \frac{1836}{2}$ for $H_2^+$ in atomic units ($m_p$ given in electron masses). As in previous exercises, we - due to numerics - use finite differences instead of derivatives and the kinetic term therefore is the [second order difference quotient](https://en.wikipedia.org/wiki/Difference_quotient#Second_order) with prefactor $-\frac{1}{2 \mu}$. The potential is given as .dat file, the minimum and therefore the equilibrium distance of the protons is roughly 2 Bohr radii.

In [ ]:
# compute eigenvals and eigenvecs, use scipy for faster algorithm
H = methods.hamiltonian(pot_even[:,1], dx)
eigenvals, eigenvecs = ssLA.eigsh(H.toarray(), k=H.shape[0])

### visualize eigenstates ###
fig, ax = plt.subplots()
ax.set_xlim((0, 10))
ax.set_ylim((-0.12, 0.12))
ax.set_title("Eigenfunction Probability Density in the |g> State")
ax.set_xlabel("x in $r_B$")
plt.ylabel("$|\psi_n(x)|^2$ in a.u.")
line, = ax.plot([], [])

def plot_eigenvec(idx):
    line.set_data(pot_even[:,0], eigenvecs[:,idx]**2 + eigenvals[idx])
    return (line,)

anim = animation.FuncAnimation(
    fig,
    plot_eigenvec,
    frames=np.arange(0, 11, 1),
    interval=500,
    blit=True
)

print("eigenvalues:", eigenvals[eigenvals <= 0.0])

plt.plot(pot_even[:,0], pot_even[:,1])
plt.close()
# save if desired
if False:
    methods.save_anim(anim, "bound_eigenstates.gif")
HTML(anim.to_jshtml())

Here are some examples of vibrational Eigenstates in the ground-state potential:
    
<img src="imgs/vibstates.png" alt="Drawing" style="width: 600px;"/>

The following plots show the influence of the grid size by arbitrarily narrowing the grid to $1/10$ and $1/5$ of the full length respectively. Clearly visible, the eigenenergies come closer to the expected null-level for $n \rightarrow \infty$.

In [ ]:
pot = pot_even[pot_even[:,0] <= 30.0]

H = methods.hamiltonian(pot[:,1], dx)
eigenvals, eigenvecs = ssLA.eigsh(H.toarray(), k=H.shape[0])

print(eigenvals.shape)

plt.figure()
plt.xlim((0, 10))
plt.ylim((-0.12, 0.12))
plt.plot(pot[:,0], pot[:,1], label="V(x) for grid size x = {}".format(30))
plt.plot(pot[:,0], eigenvecs[:,20]**2 + eigenvals[20], label="Eigenstate #20")
plt.plot(pot[:,0], eigenvecs[:,30]**2 + eigenvals[30], label="Eigenstate #30")
plt.plot(pot[:,0], eigenvecs[:,40]**2 + eigenvals[40], label="Eigenstate #40")
plt.plot(pot[:,0], eigenvecs[:,50]**2 + eigenvals[50], label="Eigenstate #50")
plt.title("Four Vibrational Eigenstates")
plt.xlabel("x in $r_B$")
plt.ylabel("$|\psi_n(x)|^2$ in a.u.")
plt.legend()

In [ ]:
pot = pot_even[pot_even[:,0] <= 60.0]

H = methods.hamiltonian(pot[:,1], dx)
eigenvals, eigenvecs = ssLA.eigsh(H.toarray(), k=H.shape[0])

print(eigenvecs.shape)

plt.figure()
plt.xlim((0, 10))
plt.ylim((-0.12, 0.12))
plt.plot(pot[:,0], pot[:,1], label="V(x) for grid size x = {}".format(50))
plt.plot(pot[:,0], eigenvecs[:,20]**2 + eigenvals[20], label="Eigenstate #20")
plt.plot(pot[:,0], eigenvecs[:,30]**2 + eigenvals[30], label="Eigenstate #30")
plt.plot(pot[:,0], eigenvecs[:,40]**2 + eigenvals[40], label="Eigenstate #40")
plt.plot(pot[:,0], eigenvecs[:,50]**2 + eigenvals[50], label="Eigenstate #50")
plt.title("Four Vibrational Eigenstates")
plt.xlabel("x in $r_B$")
plt.ylabel("$|\psi_n(x)|^2$ in a.u.")
plt.legend()

### Comparison: Description with Harmonic Oscillator

The next brief calculations compare the eigenvalues of the $H_2^+$-potential with hypothetical eigenvalues of a harmonical oscillator approximation.

<img src="imgs/harmonic.png" alt="Drawing" style="width: 400px;"/>

Theoretically we try to describe the molecule as two masses on a spring with spring constant $k = \frac{dF}{dr} = \frac{2e^2}{4\pi\epsilon_0R_e^3}$ and reduced mass $\mu = \frac{m_1m_2}{m_1 + m_2}$, wherein $R_e$ is the equilibrium distance of the protons in case of the $H_2^+$.

$$
\omega = \sqrt{\frac{k}{\mu}} \overset{\text{in at. un.}}{\approx} \sqrt{\frac{2 a_0 E_h}{(2a_0)^3}\frac{2}{1836 m_e}} \overset{m_e, E_h, a_0 = 1 \text{ in at. un.}}{=} \sqrt{\frac{1}{3672}} \approx 0.0165
$$

We therefore would obtain the vibrational eigenenergies

$$
E = \omega\left(n+\frac{1}{2}\right) \approx 0.0165\cdot n + 0.0083
$$

Which gives pretty good agreement with the numerically obtained values if one takes into considerations, that $V_{min} = -0.10267$ and therefore

$$
E \approx 0.0165\cdot n - 0.0944
$$

which is close to the $- 0.0974$ obtained numerically. The step size for the first steps is also only by a factor of $\approx 2$ different from the simplified theoretical harmonical oscillator predictions. 

## 2. Simulate the wave packet propagation without the IR laser field

Note: time in atomic units is given in unit steps of $\approx 2.4\cdot 10^{-17}s$ in SI-units.

In [ ]:
H = methods.hamiltonian(pot_even[:,1], dx)
h2_time_evolution = methods.TimeEvolution(wave_function[:,1], H.toarray())


### visualize time evolution ###
fig, ax = plt.subplots()
ax.set_xlim((0, 10))
ax.set_ylim((-0.5, 3))
ax.set_title("Propagation of Frank-Condon Wave Function in |g> State")
ax.set_xlabel("Proton Distance in Atomic Units")
ax.set_ylabel("Wavefunction [a.u.]")
line, = ax.plot([], [])

def plot_time_evolution(t):
    psit = h2_time_evolution(t)
    line.set_data(wave_function[:,0], np.abs(psit/npLA.norm(psit))**2/dx)
    return (line,)

anim = animation.FuncAnimation(
    fig,
    plot_time_evolution,
    frames=np.linspace(0, 1300, 500),
    interval=40,
    blit=True
)

plt.plot(pot_even[:,0], pot_even[:,1])
plt.close()
# save if desired
if False:
    methods.save_anim(anim, "time_evol_wo_laser.gif", fps=10)
HTML(anim.to_jshtml())

## 3. Simulate the dynamics of the time-dependent system

First, let's evaluate the coupling/ avoided-crossing effect with different coupling strengths. During laser field exposure, the coupling will vary.

<img src="imgs/avoided.png" alt="Drawing" style="width: 600px;"/>

In [ ]:
# take frequency value in atomic units from publication
omega = 0.0599

xvals = pot_even[:,0]
V1 = pot_even[:,1]
V2 = pot_uneven[:,1] - omega

### visualize time evolution ###
fig, ax = plt.subplots()
ax.set_xlim((0, 10))
ax.set_ylim((-0.12, 0.12))
ax.set_title("Avoided Crossing with $W = {0.0001,...,0.1}$")
ax.set_xlabel("x in $r_B$")
ax.set_ylabel("V(x) in $E_h$")

lines = []
for index in range(2):
    lobj = ax.plot([],[])[0]
    lines.append(lobj)

def init():
    for line in lines:
        line.set_data([],[])
    return lines

def plot_w_alteration(W):
    # solve eigenvalue equation to diagonalize matrix
    y1 = 0.5*(V1 + V2) + 0.5*np.sqrt((V1 - V2)**2 + 4*W**2)
    y2 = 0.5*(V1 + V2) - 0.5*np.sqrt((V1 - V2)**2 + 4*W**2)
    
    xlist = [xvals, xvals]
    ylist = [y1, y2]
    
    for lnum, line in enumerate(lines):
        line.set_data(xlist[lnum], ylist[lnum])
        
    return lines

anim = animation.FuncAnimation(
    fig,
    plot_w_alteration,
    init_func=init,
    frames=np.linspace(0.0001, 0.1, 50),
    interval=40,
    blit=True
)

# orignal potentials
plt.plot(xvals, V1, color="red", linestyle="dashed", label="$|g,0\hbar\omega>$")
plt.plot(xvals, V2, color="black", linestyle="dashed", label="$|u,-1\hbar\omega>$")
plt.legend()
plt.close()
# save if desired
if False:
    methods.save_anim(anim, "avoided_crossing.gif", fps=5)
HTML(anim.to_jshtml())

For better visibility, the potential is plotted multiplied with a factor 5 in the next plot. For $\tau$ the value $\tau = 1075.3$ is set in $methods.py$, which is equal to the 26 fs mentioned in [[1]](https://doi.org/10.1103/PhysRevA.93.012507).

In [ ]:
potential_scaling = 5
tsteps = 300
dt = 5

potential = methods.LaserFieldPotential(pot_even, pot_uneven, dipole_coupling)
split_step = methods.SplitStepMethod(len(wave_function), dx)

H = methods.hamiltonian(pot_even[:,1], dx)
eigenvals, eigenvecs = ssLA.eigsh(H.toarray(), k=H.shape[0])

# storage space for results
pot_t = np.zeros((tsteps+1, len(wave_function)), dtype=np.float64)
psi_t = np.zeros((tsteps+1, len(wave_function)), dtype=np.complex128)
prepared_t = np.zeros((tsteps+1, len(wave_function)), dtype=np.complex128)

# set initial values
pot_t[0,:] = potential(0)
psi_t[0,:] = wave_function[:,1]
prepared_t[0,:] = eigenvecs[:,7] + eigenvecs[:,8]

# compute time evolution
for tstep in range(1, tsteps+1):
    pot_t[tstep,:] = potential(dt * tstep)
    psi_t[tstep,:] = split_step(psi_t[tstep-1], dt, pot_t[tstep])
    prepared_t[tstep,:] = split_step(prepared_t[tstep-1], dt, prepared_t[tstep])

### visualize time evolution ###
fig, ax = plt.subplots()
ax.set_xlim((0, 20))
ax.set_ylim((-1, 3))
ax.set_title("Propagation of Frank-Condon Wave Function (orange) and mixture of \n " \
             "eigenstates #7/#8 (green) in time-dependent potential")
ax.set_xlabel("Proton Distance in Atomic Units")
ax.set_ylabel("Wavefunction [a.u.]")
line_0, = ax.plot([], [])
line_1, = ax.plot([], [])
line_2, = ax.plot([], [])

def plot_laser_field_time_evolution(idx):
    line_0.set_data(wave_function[:,0], potential_scaling * pot_t[idx])
    line_1.set_data(wave_function[:,0], np.abs(psi_t[idx,:]/npLA.norm(psi_t[idx,:]))**2/dx)
    line_2.set_data(wave_function[:,0], np.abs(prepared_t[idx,:]/npLA.norm(prepared_t[idx,:]))**2/dx)
    return (line_0, line_1, line_2)

anim = animation.FuncAnimation(
    fig,
    plot_laser_field_time_evolution,
    frames=range(0, tsteps+1),
    interval=50,
    blit=True
)

plt.close()
# save if desired
if False:
    methods.save_anim(anim, "laser_field_evolution.gif", fps=10)
HTML(anim.to_jshtml())

## 4. Scan the delay time and analyze the momentum distribution

Discussions about the chosen total time range of  $2\cdot 10^4$  can be found in the description of the project or in [[5]](https://doi.org/10.1088%2F0953-4075%2F36%2F4%2F305).

Runtime is roughly 22 minutes on my machine with Ryzen5 2400G and scales linearly in the number of taus.

In [ ]:
tsteps = 2000
dt = 10
cutoff = 120                           # cutoff at 6 Bohr radii
taus = (np.arange(400)*1 - 5)*41.34    # 41.34 conversion atomic units to fs 

potential = methods.LaserFieldPotential(pot_even, pot_uneven, dipole_coupling)
split_step = methods.SplitStepMethod(len(wave_function), dx)

H = methods.hamiltonian(pot_even[:,1], dx)
eigenvals, eigenvecs = ssLA.eigsh(H.toarray(), k=H.shape[0])

# storage space for results
momentum_psi = np.zeros((len(taus), len(wave_function) - cutoff), dtype=np.complex128)
momentum_prepared = np.zeros((len(taus), len(wave_function) - cutoff), dtype=np.complex128)

for idx, tau in enumerate(taus):
    potential.tau = tau

    # storage space for results
    pot_t = np.zeros((tsteps+1, len(wave_function)), dtype=np.float64)
    psi_t = np.zeros((tsteps+1, len(wave_function)), dtype=np.complex128)
    prepared_t = np.zeros((tsteps+1, len(wave_function)), dtype=np.complex128)

    # set initial values
    pot_t[0,:] = potential(0)
    psi_t[0,:] = wave_function[:,1]
    prepared_t[0,:] = eigenvecs[:,7] + eigenvecs[:,8]

    # compute time evolution
    for tstep in range(1, tsteps+1):
        pot_t[tstep,:] = potential(dt * tstep)
        psi_t[tstep,:] = split_step(psi_t[tstep-1], dt, pot_t[tstep])
        prepared_t[tstep,:] = split_step(prepared_t[tstep-1], dt, prepared_t[tstep])
    
    momentum_psi[idx] = np.fft.fft(psi_t[-1][cutoff:]**2)
    momentum_prepared[idx] = np.fft.fft(prepared_t[-1][cutoff:]**2)

For clarification one Fourier-analyzed spectral slice for the delay time of $\tau = 95$ fs to assess correctness of momenta.

In [ ]:
# one select spectral slice for tau = 100-5 fs
npoints = len(wave_function)-cutoff
kvals = 2*np.pi*np.fft.fftfreq(npoints, d=dx)
start = 0
stop = 5879       # total length, given by 300/0.05-120 = 5880
plt.plot(kvals[start:stop], np.abs(momentum_psi[100,start:stop])**2)
plt.title(r"Spectral Slice for $\tau = 95$ fs")
plt.xlabel("momentum (atomic units)")
plt.ylabel("intensity (arb. units)")

Next, we visualize the count distribution of FC wave packet for tau up to 395 fs.

In [ ]:
plt.imshow(np.abs(momentum_psi.T)**2, cmap=plt.cm.jet, aspect="auto",extent=[-5,395,stop,start])
plt.ylim([500,1100])
plt.xlabel(r'$\tau$ (fs)')
plt.ylabel('momentum (arb. un.)')
plt.title("Count Distribution of FC Wave Packet")
plt.colorbar()

plt.subplots_adjust(top=1, bottom=0, left=-1, right=1, hspace=0.3, wspace=0)

Similar calculation, this time for $\tau$ only up to 195 fs.

In [ ]:
tsteps = 2000
dt = 10
cutoff = 120                           # cutoff at 6 Bohr radii
taus = (np.arange(200)*1 - 5)*41.34    # 41.34 conversion atomic units to fs 

potential = methods.LaserFieldPotential(pot_even, pot_uneven, dipole_coupling)
split_step = methods.SplitStepMethod(len(wave_function), dx)

H = methods.hamiltonian(pot_even[:,1], dx)
eigenvals, eigenvecs = ssLA.eigsh(H.toarray(), k=H.shape[0])

# storage space for results
momentum_psi2 = np.zeros((len(taus), len(wave_function) - cutoff), dtype=np.complex128)
momentum_prepared2 = np.zeros((len(taus), len(wave_function) - cutoff), dtype=np.complex128)

for idx, tau in enumerate(taus):
    potential.tau = tau

    # storage space for results
    pot_t = np.zeros((tsteps+1, len(wave_function)), dtype=np.float64)
    psi_t = np.zeros((tsteps+1, len(wave_function)), dtype=np.complex128)
    prepared_t = np.zeros((tsteps+1, len(wave_function)), dtype=np.complex128)

    # set initial values
    pot_t[0,:] = potential(0)
    psi_t[0,:] = wave_function[:,1]
    prepared_t[0,:] = eigenvecs[:,7] + eigenvecs[:,8]

    # compute time evolution
    for tstep in range(1, tsteps+1):
        pot_t[tstep,:] = potential(dt * tstep)
        psi_t[tstep,:] = split_step(psi_t[tstep-1], dt, pot_t[tstep])
        prepared_t[tstep,:] = split_step(prepared_t[tstep-1], dt, prepared_t[tstep])
    
    momentum_psi2[idx] = np.fft.fft(psi_t[-1][cutoff:]**2)
    momentum_prepared2[idx] = np.fft.fft(prepared_t[-1][cutoff:]**2)

Lastly, we visualize the count distribution of FC wave packet for tau up to 195 fs.

In [ ]:
plt.imshow(np.abs(momentum_psi2.T)**2, cmap=plt.cm.jet, aspect="auto",extent=[-5,195,stop,start])
plt.ylim([500,1100])
plt.xlabel(r'$\tau$ (fs)')
plt.ylabel('momentum (arb. un.)')
plt.title("Count Distribution of FC Wave Packet")
plt.colorbar()

plt.subplots_adjust(top=1, bottom=0, left=-1, right=1, hspace=0.3, wspace=0)